# Use cases

## Preamble

In [ ]:
# Import packages
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import warnings
import inputs.parameters_and_options as inpprm
import inputs.data as inpdt

# Define file paths
path_code = '..'
path_outputs = path_code + '/Output/'
path_use_case_insur = path_outputs + 'use_case_insur/'
path_use_case_cchange = path_outputs + 'use_case_cchange/'
path_folder = path_code + '/Data/'
path_data = path_folder + 'data_Cape_Town/'
path_precalc_inp = path_folder + 'precalculated_inputs/'
path_scenarios = path_data + 'Scenarios/'
path_precalc_transp = path_folder + 'precalculated_transport/'

In [ ]:
# Load geographic data
geo_grid = gpd.read_file(path_data + 'grid_reference_500.shp')
geo_grid.to_crs(4326, inplace=True)
warnings.simplefilter("ignore", UserWarning)
geo_grid['lon'] = geo_grid.centroid.x
geo_grid['lat'] = geo_grid.centroid.y

# Load pre-processed flood damage tables

## First for the insurance use case
damage_maps_concat_insur = pd.read_csv(path_use_case_insur + 'damage_maps_concat.csv')
damage_map_compar_insur = pd.read_csv(path_use_case_insur + 'damage_map_compar.csv')
damage_maps_shareinc_insur = pd.read_csv(path_use_case_insur + 'damage_maps_shareinc_concat.csv')
damage_map_compar_shareinc_insur = pd.read_csv(path_use_case_insur + 'damage_map_compar_shareinc.csv')
equil_maps_concat_insur = pd.read_csv(path_use_case_insur + 'equil_maps_concat.csv')
equil_map_compar_insur = pd.read_csv(path_use_case_insur + 'equil_map_compar.csv')
agg_stat_rel_damages_insur = pd.read_csv(path_use_case_insur + 'agg_stat_rel_damages.csv')
utility_insur = pd.read_csv(path_use_case_insur + 'utility_insur.csv')
utility_noinsur = pd.read_csv(path_use_case_insur + 'utility_noinsur.csv')

## Then for the climate change use case
damage_maps_concat_cchange = pd.read_csv(path_use_case_cchange + 'damage_maps_concat.csv')
damage_map_compar_cchange = pd.read_csv(path_use_case_cchange + 'damage_map_compar.csv')
damage_maps_shareinc_cchange = pd.read_csv(path_use_case_cchange + 'damage_maps_shareinc_concat.csv')
damage_map_compar_shareinc_cchange = pd.read_csv(path_use_case_cchange + 'damage_map_compar_shareinc.csv')
equil_maps_concat_cchange = pd.read_csv(path_use_case_cchange + 'equil_maps_concat.csv')
equil_map_compar_cchange = pd.read_csv(path_use_case_cchange + 'equil_map_compar.csv')
agg_stat_rel_damages_cchange = pd.read_csv(path_use_case_cchange + 'agg_stat_rel_damages.csv')
utility_cchange = pd.read_csv(path_use_case_cchange + 'utility_cchange.csv')
utility_nocchange = pd.read_csv(path_use_case_cchange + 'utility_nocchange.csv')

# Rearrange tables for plots

## First for the insurance use case
damage_map_insur = damage_maps_concat_insur.loc[damage_maps_concat_insur['insur']=='Yes'].copy()
damage_map_noinsur = damage_maps_concat_insur.loc[damage_maps_concat_insur['insur']=='No'].copy()
damage_map_shareinc_insur = damage_maps_shareinc_insur.loc[damage_maps_shareinc_insur['insur']=='Yes'].copy()
damage_map_shareinc_noinsur = damage_maps_shareinc_insur.loc[damage_maps_shareinc_insur['insur']=='No'].copy()
equil_map_insur = equil_maps_concat_insur.loc[equil_maps_concat_insur['insur']=='Yes'].copy()
equil_map_noinsur = equil_maps_concat_insur.loc[equil_maps_concat_insur['insur']=='No'].copy()

## Then for the climate change use case
damage_map_cchange = damage_maps_concat_cchange.loc[damage_maps_concat_cchange['cchange']=='Yes'].copy()
damage_map_nocchange = damage_maps_concat_cchange.loc[damage_maps_concat_cchange['cchange']=='No'].copy()
damage_map_shareinc_cchange = damage_maps_shareinc_cchange.loc[damage_maps_shareinc_cchange['cchange']=='Yes'].copy()
damage_map_shareinc_nocchange = damage_maps_shareinc_cchange.loc[damage_maps_shareinc_cchange['cchange']=='No'].copy()
equil_map_cchange = equil_maps_concat_cchange.loc[equil_maps_concat_cchange['cchange']=='Yes'].copy()
equil_map_nocchange = equil_maps_concat_cchange.loc[equil_maps_concat_cchange['cchange']=='No'].copy()

In [ ]:
# We also import some input data for reference

## Import parameters and options
options = inpprm.import_options()
param = inpprm.import_param(
    path_precalc_inp, options)

## Import land use + amenity data

(interest_rate, population, housing_type_data, total_RDP
 ) = inpdt.import_macro_data(param, path_scenarios, path_folder)
grid, center = inpdt.import_grid(path_data)
(data_rdp, housing_types_sp, data_sp, mitchells_plain_grid_baseline,
 grid_formal_density_HFA, threshold_income_distribution, income_distribution,
 cape_town_limits) = inpdt.import_households_data(path_precalc_inp)
housing_types_data = pd.read_excel(path_folder + 'housing_types_grid_sal.xlsx')
housing_types_data[np.isnan(housing_types_data)] = 0

(spline_RDP, spline_estimate_RDP, spline_land_RDP,
 spline_land_backyard, spline_land_informal, spline_land_constraints,
 number_properties_RDP) = (
     inpdt.import_land_use(grid, options, param, data_rdp, housing_types_data,
                           housing_type_data, path_data, path_folder)
)

coeff_land = inpdt.import_coeff_land(
    spline_land_constraints, spline_land_backyard, spline_land_informal,
    spline_land_RDP, param, 0)

amenities = inpdt.import_amenities(path_precalc_inp, options)

income_net_of_commuting_costs = np.load(
    path_precalc_transp + 'GRID_incomeNetOfCommuting_0.npy')

In [ ]:
# We re-arrange absolute damage tables in a specific format for aggregate bar plots

warnings.simplefilter("ignore", RuntimeWarning)

## We define dimension names
yes_no = ['Yes', 'No']
housing_types = ['formal', 'subsidized', 'informal', 'backyard']
flood_types = ['fluvialu', 'pluvial', 'coastal']
damage_types = ['structure', 'content']


## We process absolute damages in the insurance use case

dict_agg_damages_insur = {}

for indic in yes_no:
    for flood in flood_types:
        for housing in housing_types:
            for damage in damage_types:
                dict_agg_damages_insur[
                    indic + '_' + flood + '_' + housing + '_' + damage
                ] = damage_maps_concat_insur.loc[
                    damage_maps_concat_insur['insur'] == indic,
                    flood + '_' + housing + '_' + damage].sum()

for flood in flood_types:
    for housing in housing_types:
        for damage in damage_types:
            dict_agg_damages_insur[
                'pct_change_' + flood + '_' + housing + '_' + damage
            ] = (dict_agg_damages_insur[
                'No_' + flood + '_' + housing + '_' + damage]
                / dict_agg_damages_insur[
                'Yes_' + flood + '_' + housing + '_' + damage]
                - 1)

dict_df_agg_damages_insur = {}

for flood in flood_types:

    df_agg_damages_insur = pd.DataFrame()
    df_agg_damages_insur.at[0, 'Structures'] = (
        dict_agg_damages_insur['Yes_' + flood + '_formal_structure'])
    df_agg_damages_insur.at[0, 'Contents'] = (
        dict_agg_damages_insur['Yes_' + flood + '_formal_content'])
    df_agg_damages_insur.at[1, 'Structures'] = (
        dict_agg_damages_insur['Yes_' + flood + '_subsidized_structure'])
    df_agg_damages_insur.at[1, 'Contents'] = (
        dict_agg_damages_insur['Yes_' + flood + '_subsidized_content'])
    df_agg_damages_insur.at[2, 'Structures'] = (
        dict_agg_damages_insur['Yes_' + flood + '_informal_structure'])
    df_agg_damages_insur.at[2, 'Contents'] = (
        dict_agg_damages_insur['Yes_' + flood + '_informal_content'])
    df_agg_damages_insur.at[3, 'Structures'] = (
        dict_agg_damages_insur['Yes_' + flood + '_backyard_structure'])
    df_agg_damages_insur.at[3, 'Contents'] = (
        dict_agg_damages_insur['Yes_' + flood + '_backyard_content'])

    df_agg_damages_insur.rename(index={
        0: 'Formal private', 1: 'Formal subsidized', 2: 'Informal settlements',
        3: 'Informal backyards'}, inplace=True)
    df_agg_damages_insur = df_agg_damages_insur / 1000000

    df_agg_damages_noinsur = pd.DataFrame()
    df_agg_damages_noinsur.at[0, 'Structures'] = (
        dict_agg_damages_insur['No_' + flood + '_formal_structure'])
    df_agg_damages_noinsur.at[0, 'Contents'] = (
        dict_agg_damages_insur['No_' + flood + '_formal_content'])
    df_agg_damages_noinsur.at[1, 'Structures'] = (
        dict_agg_damages_insur['No_' + flood + '_subsidized_structure'])
    df_agg_damages_noinsur.at[1, 'Contents'] = (
        dict_agg_damages_insur['No_' + flood + '_subsidized_content'])
    df_agg_damages_noinsur.at[2, 'Structures'] = (
        dict_agg_damages_insur['No_' + flood + '_informal_structure'])
    df_agg_damages_noinsur.at[2, 'Contents'] = (
        dict_agg_damages_insur['No_' + flood + '_informal_content'])
    df_agg_damages_noinsur.at[3, 'Structures'] = (
        dict_agg_damages_insur['No_' + flood + '_backyard_structure'])
    df_agg_damages_noinsur.at[3, 'Contents'] = (
        dict_agg_damages_insur['No_' + flood + '_backyard_content'])

    df_agg_damages_noinsur.rename(index={
        0: 'Formal private', 1: 'Formal subsidized', 2: 'Informal settlements',
        3: 'Informal backyards'}, inplace=True)
    df_agg_damages_noinsur = df_agg_damages_noinsur / 1000000

    dict_df_agg_damages_insur[flood + '_insur'] = df_agg_damages_insur
    dict_df_agg_damages_insur[flood + '_noinsur'] = df_agg_damages_noinsur


## We do the same in the climate change use case

dict_agg_damages_cchange = {}
for cc_indic in yes_no:
    for flood in flood_types:
        for housing in housing_types:
            for damage in damage_types:
                dict_agg_damages_cchange[
                    cc_indic + '_' + flood + '_' + housing + '_' + damage
                ] = damage_maps_concat_cchange.loc[
                    damage_maps_concat_cchange['cchange'] == cc_indic,
                    flood + '_' + housing + '_' + damage].sum()

for flood in flood_types:
    for housing in housing_types:
        for damage in damage_types:
            dict_agg_damages_cchange[
                'pct_change_' + flood + '_' + housing + '_' + damage
            ] = (dict_agg_damages_cchange[
                'Yes_' + flood + '_' + housing + '_' + damage]
                / dict_agg_damages_cchange[
                'No_' + flood + '_' + housing + '_' + damage]
                - 1)

dict_df_agg_damages_cchange = {}

for flood in flood_types:

    df_agg_damages_cchange = pd.DataFrame()
    df_agg_damages_cchange.at[0, 'Structures'] = (
        dict_agg_damages_cchange['Yes_' + flood + '_formal_structure'])
    df_agg_damages_cchange.at[0, 'Contents'] = (
        dict_agg_damages_cchange['Yes_' + flood + '_formal_content'])
    df_agg_damages_cchange.at[1, 'Structures'] = (
        dict_agg_damages_cchange['Yes_' + flood + '_subsidized_structure'])
    df_agg_damages_cchange.at[1, 'Contents'] = (
        dict_agg_damages_cchange['Yes_' + flood + '_subsidized_content'])
    df_agg_damages_cchange.at[2, 'Structures'] = (
        dict_agg_damages_cchange['Yes_' + flood + '_informal_structure'])
    df_agg_damages_cchange.at[2, 'Contents'] = (
        dict_agg_damages_cchange['Yes_' + flood + '_informal_content'])
    df_agg_damages_cchange.at[3, 'Structures'] = (
        dict_agg_damages_cchange['Yes_' + flood + '_backyard_structure'])
    df_agg_damages_cchange.at[3, 'Contents'] = (
        dict_agg_damages_cchange['Yes_' + flood + '_backyard_content'])

    df_agg_damages_cchange.rename(index={
        0: 'Formal private', 1: 'Formal subsidized', 2: 'Informal settlements',
        3: 'Informal backyards'}, inplace=True)
    df_agg_damages_cchange = df_agg_damages_cchange / 1000000

    df_agg_damages_nocchange = pd.DataFrame()
    df_agg_damages_nocchange.at[0, 'Structures'] = (
        dict_agg_damages_cchange['No_' + flood + '_formal_structure'])
    df_agg_damages_nocchange.at[0, 'Contents'] = (
        dict_agg_damages_cchange['No_' + flood + '_formal_content'])
    df_agg_damages_nocchange.at[1, 'Structures'] = (
        dict_agg_damages_cchange['No_' + flood + '_subsidized_structure'])
    df_agg_damages_nocchange.at[1, 'Contents'] = (
        dict_agg_damages_cchange['No_' + flood + '_subsidized_content'])
    df_agg_damages_nocchange.at[2, 'Structures'] = (
        dict_agg_damages_cchange['No_' + flood + '_informal_structure'])
    df_agg_damages_nocchange.at[2, 'Contents'] = (
        dict_agg_damages_cchange['No_' + flood + '_informal_content'])
    df_agg_damages_nocchange.at[3, 'Structures'] = (
        dict_agg_damages_cchange['No_' + flood + '_backyard_structure'])
    df_agg_damages_nocchange.at[3, 'Contents'] = (
        dict_agg_damages_cchange['No_' + flood + '_backyard_content'])

    df_agg_damages_nocchange.rename(index={
        0: 'Formal private', 1: 'Formal subsidized', 2: 'Informal settlements',
        3: 'Informal backyards'}, inplace=True)
    df_agg_damages_nocchange = df_agg_damages_nocchange / 1000000

    dict_df_agg_damages_cchange[flood + '_cchange'] = df_agg_damages_cchange
    dict_df_agg_damages_cchange[flood + '_nocchange'] = df_agg_damages_nocchange

## Flood risks

### Insurance policy

#### Introduction

Our model allows us to compare two polar cases: one in which agents (both on the supply and demand sides of  the housing market) perfectly anticipate flood risks (based on existing global flood maps), and the other where they do not at all take them into account. Note that anticipations here can be understood as a mechanism of self-insurance whereby agents perfectly smooth consumption across states of the world by systematically paying for the expected cost of flood damages in any given year: we recall that ours is a static equilibrium model calibrated for baseline year 2011 (for which census data is available).

It can be shown that self-insurance is a substitute for market insurance: our use case therefore boils down to comparative statics between perfect risk-based insurance - while remaining agnostic about the practical mix of schemes involved - and no insurance. It can also be shown that other ex-ante flood managements strategies - namely zoning and subsidized insurance - are second-best options that are easier to implement and less technically demanding, yet yield similar welfare gains. This adds to the generality of our findings.

We want to make clear from the outset that we do not aim at explicitly modelling insurance choice and/or belief formation, but rather at showing their distributional welfare consequences within an urban economics framework that converges towards a long-run equilibrium. The current reality may be expressed as a convex combination of our two polar cases, and the difference in outcomes between the two should be interpreted as an upper bound on the achievable welfare gains/losses from implementing the corresponding policy. Also note that protection investments (either self-provided, market-based, or state-funded) are not modelled here and would require a separate use case, as they act like a complement to insurance.

That being said, let us dig into the insurance case.

#### Inputs

Our model allows us to distinguish between fluvial, pluvial, and coastal flood risks. Fluvial and pluvial flood risks are based upon FATHOM global flood maps, and coastal flood risks upon DELTARES global flood maps. Those maps are merged with the *City of Cape Town* (CoCT) analysis grid to obtain estimates of flood-prone area share and maximum flood depth levels for distinct return periods (corresponding to probabilities of ocurrence in any given year), at the grid cell level (500x500m). For reference, the climate conditions used to produce the flood maps are the ones prevailing in 2018, which we assume to be roughly in line with the ones prevailing at our baseline year. Also, our geographic units are 30 times bigger than the units used in the raw flood maps, hence we consider them as being of good enough quality for what we define as the "local" level. Then, this information is converted into an expected fraction of capital destroyed, leveraging estimates from the insurance literature. This measure varies across damage type (either building structures or good contents stored at home) and building materials (hence housing types) for damages on structures. The spatial distribution of such depreciation rate is shown below for damages on contents (which do not vary across housing types).

In [ ]:
# Note that capital depreciation does not change across scenarios
equil_map_insur.loc[equil_map_insur['flood_deprec_content_formal'] == 0, 'flood_deprec_content_formal'] = np.nan
fig = px.choropleth_mapbox(
    equil_map_insur,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='flood_deprec_content_formal',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-terrain',
    opacity=0.5,
    labels={'locations': 'Pixel ID', 'lon': 'Lon.', 'lat': 'Lat.',
            'flood_deprec_content_formal': '% depreciation'},
    title='Estimated fraction of capital destroyed due to floods'
    + ' (content damages)',
    color_continuous_scale="Reds",
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'flood_deprec_content_formal': ':.2f'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

As could be expected, flood exposure follows the coastline (for coastal flood risks), waterways (for fluvial flood risks), and topological depression areas (for pluvial flood risks). The distribution of its intensity is not obvious a priori however. Note that this is only a first step towards quantifying damages, as their final value also depends on the housing choices made by households: highly exposed but low-density areas such as the West front of the peninsula should feature relatively low damages. Also note that the flood maps used to calibrate this variable are so-called "undefended" flood maps. This means that they do not take into account existing infrastructure that may affect the spatial distribution of flood risks (as assessing this dimension on a global scale would provide noisy estimates at best). This is why we prefer to intepret our results in changes, rather than in levels: all else equal (including existing infrastructure), what is the impact of a change on a given dimension (for instance, insurance) over outcomes of the model? 

In our model, geography is also defined by several other dimensions, namely land availability per housing type, location-specific amenities (with an added disamenity factor for informal housing), and calibrated income net of commuting costs (accounting for distribution of commuters across job centers for different income groups). Although they do not play a key role in this specific use case, we plot some of them below for reference as they constitute the universe of exogenous factors that influence households' endogenous location decisions.

In [ ]:
amenities_df = pd.DataFrame({'amenities': amenities})
amenities_df['lon'] = geo_grid.lon
amenities_df['lat'] = geo_grid.lat
amenities_df.loc[
    (coeff_land[0, :] == 0) & (coeff_land[1, :] == 0)
    & (coeff_land[2, :] == 0) & (coeff_land[3, :] == 0), 'amenities'] = np.nan

fig = px.choropleth_mapbox(
    amenities_df,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='amenities',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-terrain',
    opacity=0.5,
    labels={'locations': 'Pixel ID', 'lon': 'Lon.', 'lat': 'Lat.',
            'amenities': 'Amenity index'},
    title='Estimated amenity index (in habitable areas)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=1,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'amenities': ':.2f'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

As expected, we find high amenity values in the city centre and in specific areas well-endowed by nature.

In [ ]:
income_net_of_commuting_costs_df = pd.DataFrame(
    {'1': income_net_of_commuting_costs[0, :],
     '2': income_net_of_commuting_costs[1, :],
     '3': income_net_of_commuting_costs[2, :],
     '4': income_net_of_commuting_costs[3, :]})
income_net_of_commuting_costs_df.loc[
    np.isnan(amenities_df['amenities']), '1'] = np.nan
income_net_of_commuting_costs_df.loc[
    np.isnan(amenities_df['amenities']), '2'] = np.nan
income_net_of_commuting_costs_df.loc[
    np.isnan(amenities_df['amenities']), '3'] = np.nan
income_net_of_commuting_costs_df.loc[
    np.isnan(amenities_df['amenities']), '4'] = np.nan
income_net_of_commuting_costs_df['lon'] = geo_grid.lon
income_net_of_commuting_costs_df['lat'] = geo_grid.lat

In [ ]:
fig = px.choropleth_mapbox(
    income_net_of_commuting_costs_df,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='1',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-terrain',
    opacity=0.5,
    labels={'locations': 'Pixel ID', 'lon': 'Lon.', 'lat': 'Lat.',
            '1': 'Net income'},
    title='Theoretical annual income net of commuting costs for income group 1'
    + ' (in habitable areas)',
    color_continuous_scale="Reds",
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                '1': ':,.0f'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

In [ ]:
fig = px.choropleth_mapbox(
    income_net_of_commuting_costs_df,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='2',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-terrain',
    opacity=0.5,
    labels={'locations': 'Pixel ID', 'lon': 'Lon.', 'lat': 'Lat.',
            '2': 'Net income'},
    title='Estimated annual income net of commuting costs for income group 2'
    + ' (in habitable areas)',
    color_continuous_scale="Reds",
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                '2': ':,.0f'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

In [ ]:
fig = px.choropleth_mapbox(
    income_net_of_commuting_costs_df,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='3',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-terrain',
    opacity=0.5,
    labels={'locations': 'Pixel ID', 'lon': 'Lon.', 'lat': 'Lat.',
            '3': 'Net income'},
    title='Estimated annual income net of commuting costs for income group 3'
    + ' (in habitable areas)',
    color_continuous_scale="Reds",
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                '3': ':,.0f'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

In [ ]:
fig = px.choropleth_mapbox(
    income_net_of_commuting_costs_df,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='4',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-terrain',
    opacity=0.5,
    labels={'locations': 'Pixel ID', 'lon': 'Lon.', 'lat': 'Lat.',
            '4': 'Net income'},
    title='Estimated annual income net of commuting costs for income group 4'
    + ' (in habitable areas)',
    color_continuous_scale="Reds",
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                '4': ':,.0f'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

As expected from a gravity-like commuting choice model, we find a negative income gradient around job centres, which correspond to heat peakpoints on the maps. Also note that job centre (and income) distribution becomes less dispersed as we climb the job ladder.

Finally, note that even if our model allows us to make predictions at a very granular level, no validation data exists at such a fine level of aggregation. This means that, even though we get good performance when aggregating outcomes at the Small Place (SP) or Small Area Level (SAL), one should be cautious when interpreting results for an isolated grid cell.

#### Aggregate damages

We refer the reader to the technical documentation for a complete review of how the model works. Let us focus here on the results in terms of flood damages. We plot below the distribution of aggregate annual damages across housing and damage types, comparing our two polar cases, for each flood type. 

In [ ]:
# We store new names to update individual plots before supeirmposing them
newnames_noinsur = {'Structures': 'Structures (w/o/ insurance)',
                    'Contents': 'Contents (w/o/ insurance)'}
newnames_insur = {'Structures': 'Structures (w/ insurance)',
                  'Contents': 'Contents (w/ insurance)'}

In [ ]:
# First for fluvial floods

fig_fluvialu_noinsur = px.bar(
    dict_df_agg_damages_insur['fluvialu_noinsur'],
    x=dict_df_agg_damages_insur['fluvialu_noinsur'].index,
    y=dict_df_agg_damages_insur['fluvialu_noinsur'].columns,
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Pastel1,
    title='Estimated annual damages from fluvial floods (in M rands, 2011)',
    labels={'index': 'Housing type', 'value': 'Annual damages',
            'variable': 'Damage type'},
    opacity=0.8,
    template='plotly_white')

fig_fluvialu_noinsur.for_each_trace(
    lambda t: t.update(
        name=newnames_noinsur[t.name],
        legendgroup=newnames_noinsur[t.name],
        hovertemplate=t.hovertemplate.replace(
         t.name, newnames_noinsur[t.name])
    ))

fig_fluvialu_insur = px.bar(
    dict_df_agg_damages_insur['fluvialu_insur'],
    x=dict_df_agg_damages_insur['fluvialu_insur'].index,
    y=dict_df_agg_damages_insur['fluvialu_insur'].columns,
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Set1,
    title='Estimated annual damages from fluvial floods (in M rands, 2011)',
    labels={'index': 'Housing type', 'value': 'Annual damages',
            'variable': 'Damage type'},
    opacity=0.8,
    template='plotly_white')

fig_fluvialu_insur.for_each_trace(
    lambda t: t.update(
        name=newnames_insur[t.name],
        legendgroup=newnames_insur[t.name],
        hovertemplate=t.hovertemplate.replace(
         t.name, newnames_insur[t.name])
    ))

fig_fluvialu = go.Figure(fig_fluvialu_noinsur)
fig_fluvialu = fig_fluvialu.add_traces(fig_fluvialu_insur.data)
fig_fluvialu.show()

In [ ]:
# Then for pluvial floods

fig_pluvial_noinsur = px.bar(
    dict_df_agg_damages_insur['pluvial_noinsur'],
    x=dict_df_agg_damages_insur['pluvial_noinsur'].index,
    y=dict_df_agg_damages_insur['pluvial_noinsur'].columns,
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Pastel1,
    title='Estimated annual damages from pluvial floods (in M rands, 2011)',
    labels={'index': 'Housing type', 'value': 'Annual damages',
            'variable': 'Damage type'},
    opacity=0.8,
    template='plotly_white')

fig_pluvial_noinsur.for_each_trace(
    lambda t: t.update(
        name=newnames_noinsur[t.name],
        legendgroup=newnames_noinsur[t.name],
        hovertemplate=t.hovertemplate.replace(
         t.name, newnames_noinsur[t.name])
    ))

fig_pluvial_insur = px.bar(
    dict_df_agg_damages_insur['pluvial_insur'],
    x=dict_df_agg_damages_insur['pluvial_insur'].index,
    y=dict_df_agg_damages_insur['pluvial_insur'].columns,
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Set1,
    title='Estimated annual damages from pluvial floods (in M rands, 2011)',
    labels={'index': 'Housing type', 'value': 'Annual damages',
            'variable': 'Damage type'},
    opacity=0.8,
    template='plotly_white')

fig_pluvial_insur.for_each_trace(
    lambda t: t.update(
        name=newnames_insur[t.name],
        legendgroup=newnames_insur[t.name],
        hovertemplate=t.hovertemplate.replace(
         t.name, newnames_insur[t.name])
    ))

fig_pluvial = go.Figure(fig_pluvial_noinsur)
fig_pluvial = fig_pluvial.add_traces(fig_pluvial_insur.data)
fig_pluvial.show()

In [ ]:
# Finally for coastal floods

fig_coastal_noinsur = px.bar(
    dict_df_agg_damages_insur['coastal_noinsur'],
    x=dict_df_agg_damages_insur['coastal_noinsur'].index,
    y=dict_df_agg_damages_insur['coastal_noinsur'].columns,
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Pastel1,
    title='Estimated annual damages from coastal floods (in M rands, 2011)',
    labels={'index': 'Housing type', 'value': 'Annual damages',
            'variable': 'Damage type'},
    opacity=0.8,
    template='plotly_white')

fig_coastal_noinsur.for_each_trace(
    lambda t: t.update(
        name=newnames_noinsur[t.name],
        legendgroup=newnames_noinsur[t.name],
        hovertemplate=t.hovertemplate.replace(
         t.name, newnames_noinsur[t.name])
    ))

fig_coastal_insur = px.bar(
    dict_df_agg_damages_insur['coastal_insur'],
    x=dict_df_agg_damages_insur['coastal_insur'].index,
    y=dict_df_agg_damages_insur['coastal_insur'].columns,
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Set1,
    title='Estimated annual damages from coastal floods (in M rands, 2011)',
    labels={'index': 'Housing type', 'value': 'Annual damages',
            'variable': 'Damage type'},
    opacity=0.8,
    template='plotly_white')

fig_coastal_insur.for_each_trace(
    lambda t: t.update(
        name=newnames_insur[t.name],
        legendgroup=newnames_insur[t.name],
        hovertemplate=t.hovertemplate.replace(
         t.name, newnames_insur[t.name])
    ))

fig_coastal = go.Figure(fig_coastal_noinsur)
fig_coastal = fig_coastal.add_traces(fig_coastal_insur.data)
fig_coastal.show()

Several comments are in order. First, damages in formal housing are consistently bigger than in informal housing, and damages on structures also tend to outweigh damages on contents. This is because capital value of informal settlements / backyards is calibrated based on construction costs and is fairly low (hence the relative importance of contents in this category), whereas capital value of formal private housing (that is capital-intensive and can be built on several floors) is endogenously determined through supply and demand, and can get pretty high in dense locations. The bulk of overall population lives in formal private housing too, hence the observed results in spite of informal settlements / backyards being less protected from floods. Besides, informal housing dwelling units do not appear to be systematically located in zones with high flood exposure, contrary to what one may think at first. The capital value of formal subsidized housing structures is also calibrated based on construction costs but is typically higher than for informal housing. Finally, the population living in formal private housing is essentially made up of households belonging to the two richest income groups, with the two poorest concentrating in the other housing categories. Their higher amount of income available for consumption explains the relative importance of content damages in this category.

Second, damages are higher in the no-insurance case than in the perfect-insurance case. This is to be expected given optimality conditions. The insurance margin also seems to play a relatively more important role in the formal private housing sector. This can be explained by the fact that it is the least constrained of the housing submarkets, either in terms of available locations, or supply and demand. What is more surprising is the differential role insurance seems to play across flood types. Again, this can be interpreted in terms of existing constraints. Pluvial flood zones are typically more dispersed than fluvial or coastal flood zones, hence are harder to avoid (the extensive margin). Independently, coastal flood zones are typically more valuable, hence a floor below which demand would not fall (the intensive margin).

Third, damages also significantly vary in their magnitude across flood types:
- Total fluvial damages amount to roughly 280M ZAR (2011) with insurance and 570M ZAR (2011) without insurance (**+104%**)
- Total pluvial damages amount to roughly 145M ZAR (2011) with insurance and 165M ZAR (2011) without insurance (**+14%**)
- Total coastal damages amount to roughly 50M ZAR (2011) with insurance and 75M ZAR (2011) without insurance (**+50%**)

Using an inflation rate of 64% over the 2011-2021 period, and an exchange rate of 14.78 ZAR/USD, we get the following:
- Total fluvial damages amount to roughly 31M USD (2021) with insurance and 63M USD (2021) without insurance
- Total pluvial damages amount to roughly 16M USD (2021) with insurance and 18M USD (2021) without insurance
- Total coastal damages amount to roughly 6M USD (2021) with insurance and 8M USD (2021) without insurance

For comparison, a 2021 report by the World Bank ("Flood Risk and Resilience of Coastal Cities in Sub-Saharan Africa") gives values of 16M USD for rainwater (both fluvial and pluvial) flood damages, and 370,000 USD for coastal flood damages. The difference between the two estimations is essentially a question of methodology. Let us start by saying that, due to partial overlap, an exact estimation of rainwater flood damages cannot just be obtained as the sum of fluvial and pluvial flood damages in our case. Taking the maximum between the two in each grid cell, we obtain a value of 389M ZAR (2011) / 43M USD (2021) with insurance, and 675M ZAR (2011) / 75M USD (2021) without insurance (+74%). This is still a substantial overestimation for both flood types, especially coastal.

Note that the report uses essentially the same flood maps as ours for rainwater flood risks, as well as the same Extreme Sea Level (ESL) inputs for coastal flood risks, albeit not the same Digital Elevation Model (DEM) used to spatialize discrete inputs. This should not generate a dramatic difference however. Urban exposure is based upon 2015 data, which we deem close enough to our 2011 baseline, and all construction is also considered as residential, since this is the most common building type. On that point, it is worth noting that both approaches only yield estimates of direct flood costs, and not indirect ones such as transport disruption, business losses, or loss of family income (which are shown to be important in the World Bank 2019 "Lifelines" report).

Damage vulnerability curves (used to translate maximum flood depth into fraction of capital destroyed) are slightly different, but also in the same range. On that point, we actually think that we are improving on the report's estimates, since we are using different functions to account for the variety of building materials used in structures, and the separate exposure of house contents. Besides, our depth-damage functions for structures were calibrated for the African context (as in the report), and the functions for contents were calibrated for the South African context more specifically, taking into account the cap on damageable share that is advocated for in the report.

The key difference lies in the way the underlying value of vulnerable assets is estimated. The report uses satellite imagery to determine the extent of urban exposure, and multiplies this area by a (unique) maximum unitary damage value per m². Final damage estimates are then obtained by applying the depth-damage function to this total maximum value in each location . In contrast, urban extent and unitary value of underlying structures and contents are endogenous outcomes in our model: the city can be built on several floors, underlying values are heterogeneous, and can become very high in valuable areas. In fact, for a similar urban coverage, we find a total capital value for formal private housing structures that is very close to their estimate of the total exposed capital value of the city: since this is the housing category that drives most of the damages in absolute terms, our differences in results boil down to the way those values are distributed across the city in our model (they are typically high along the coast and in some flood-prone zones where there is high demand) and the value of associated contents, which we think is underestimated in the report.

To give an element of comparison, the maximum unitary damage value they consider in the report (24.6 USD, 2021) is almost the same as the one we calibrate for the capital value of one surface unit of an informal "shack" (23.78 USD, 2021). We think this is too low to reflect the average capital value of the city, and one would find damage values even bigger than our estimates by using the same methodology with a higher reference point, such as the one taken for South Africa in the 2017 JRC technical report ("Global flood depth-damage functions") that they cite (765 EUR, 2010 ; that is 1,018 USD, 2021 with 12% inflation and a 0.845 exchange rate). We therefore consider to be in the middle range.

#### Spatial distribution of absolute damages

We now look at how those absolute damages are distributed across space.

First, in the full-insurance case:

In [ ]:
damage_map_insur['lon'] = geo_grid.lon
damage_map_insur['lat'] = geo_grid.lat
damage_map_insur.loc[
    damage_map_insur['max_val'] == 0, 'max_val'] = np.nan

fig = px.choropleth_mapbox(
    damage_map_insur,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='max_val',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'max_val': 'Total', 'locations': 'Pixel ID',
            'flood_type': 'Flood type',
            'damage_formal': 'Formal private',
            'damage_subsidized': 'Formal subsidized',
            'damage_informal': 'Informal settlements',
            'damage_backyard': 'Informal backyards',
            'content_share_formal': '% of content (formal)',
            'content_share_subsidized': '% of content (subsidized)',
            'content_share_informal': '% of content (informal)',
            'content_share_backyard': '% of content (backyard)'},
    title='Estimated annual flood damages (in rands, 2011)',
    color_continuous_scale="Reds",
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f', 'flood_type': True,
                'damage_formal': ':,.0f',
                'content_share_formal': ':.0%',
                'damage_subsidized': ':,.0f',
                'content_share_subsidized': ':.0%',
                'damage_informal': ':,.0f',
                'content_share_informal': ':.0%',
                'damage_backyard': ':,.0f',
                'content_share_backyard': ':.0%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

Then, in the no-insurance case:

In [ ]:
damage_map_noinsur['lon'] = geo_grid.lon
damage_map_noinsur['lat'] = geo_grid.lat
damage_map_noinsur.loc[
    damage_map_noinsur['max_val'] == 0, 'max_val'] = np.nan

fig = px.choropleth_mapbox(
    damage_map_noinsur,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='max_val',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'max_val': 'Total', 'locations': 'Pixel ID',
            'flood_type': 'Flood type',
            'damage_formal': 'Formal private',
            'damage_subsidized': 'Formal subsidized',
            'damage_informal': 'Informal settlements',
            'damage_backyard': 'Informal backyards',
            'content_share_formal': '% of content (formal)',
            'content_share_subsidized': '% of content (subsidized)',
            'content_share_informal': '% of content (informal)',
            'content_share_backyard': '% of content (backyard)'},
    title='Estimated annual flood damages (in rands, 2011)',
    color_continuous_scale="Reds",
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f', 'flood_type': True,
                'damage_formal': ':,.0f',
                'content_share_formal': ':.0%',
                'damage_subsidized': ':,.0f',
                'content_share_subsidized': ':.0%',
                'damage_informal': ':,.0f',
                'content_share_informal': ':.0%',
                'damage_backyard': ':,.0f',
                'content_share_backyard': ':.0%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

Finally, comparing the two:

In [ ]:
damage_map_compar_insur.loc[
    damage_map_compar_insur['max_val'] == 0, 'max_val'] = np.nan

fig = px.choropleth_mapbox(
    damage_map_compar_insur,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='max_val',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'max_val': 'Total change', 'max_val_pct': '% change',
            'locations': 'Pixel ID',
            'flood_type': 'Flood type',
            'damage_formal': 'Formal private',
            'damage_formal_pct': '% change (formal)',
            'damage_subsidized': 'Formal subsidized',
            'damage_subsidized_pct': '% change (subsidized)',
            'damage_informal': 'Informal settlements',
            'damage_informal_pct': '% change (informal)',
            'damage_backyard': 'Informal backyards',
            'damage_backyard_pct': '% change (backyards)'},
    title='Flood damage surplus from no insurance (in rands, 2011)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f', 'flood_type': True,
                'damage_formal': ':,.0f', 'damage_formal_pct': ':+.0%',
                'damage_subsidized': ':,.0f', 'damage_subsidized_pct': ':+.0%',
                'damage_informal': ':,.0f', 'damage_informal_pct': ':+.0%',
                'damage_backyard': ':,.0f', 'damage_backyard_pct': ':+.0%',
                'max_val': True, 'max_val_pct': ':+.0%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

As expected, damages are higher in the no-insurance case. In both cases, their spatial distribution roughly follows the distribution of flood exposure in populated areas (as will be made clear below). Also, the adjustment between the two essentially occurs in fluvial flood zones and some coastal ones, in the formal private sector.

#### Spatial distribution of relative damages

What is not apparent in the above figures are the welfare consequences of flood risks. Due to the spatial indifference condition (that holds in equilibrium), all households belonging to the same income group share a common utility level in the full-insurance case (with higher values as we climb the income ladder). Ex-ante, the same applies to the no-insurance case, with utility levels that are higher for all income groups since households live in a no-flood world. Ex-post, the discovery of the flood damages associated with their location and housing choices differentially affects their utilities (which will be typically lower than in the full-insurance case for exposed households): the spatial indifference condition does not hold anymore as we do not allow households to sort again.

It is worth noting that, since utility is an ordinal (as opposed to cardinal) concept (at least for non-quasilinear functions), we cannot quantitatively interpret the difference in utility levels across scenarios: we may simply say that some households are made worse off or better off, but not by how much. In theory, welfare comparison is done by comparing (consumer) surplus across scenarios. However, this requires to fully back up supply and demand functions, which is often impossible without strong assumptions. What is usually done in such case is to come up with an income compensation scheme  (either through compensating or equivalent variations) that would make households indifferent across scenarios. Conditional on residential location, (group-specific) income net of commuting costs is a state variable that only affects utility through the budget constraint (as opposed to consumption choices on housing and composite good): we therefore consider it to be the relevant margin for a compensation scheme. Still, since we do not have an analytical solution for utilities (which are determined as a simulation outcome), we would need to rewrite a (computationally intensive) algorithm to optimize over the values of income net of commuting costs (that vary across income group and city space) needed to reach a target utility level, given initial conditions.

Also note that, in this specific case, we would need to compute ex-post utilities that vary at the local level in the no-insurance scenario. Given the granularity of our framework, this is not possible without a significant error margin. Besides, since we consider non-homothetic preferences with taste shocks, change in utilities does not directly identify equivalent (or compensating) variation, but only approximates it up to the first order. Since we are not studying infinitesimal changes here, we do not consider it to be a satisfying approximation.

What we can do instead is to focus on direct welfare impacts through the gain/loss in net income, that abstract from general equilibrium effects (changes in location and consumption choices across scenarios) and only consider damage costs: this is what we plot below for different housing types (that have different flood vulnerability), between the full-insurance (taken as baseline) and the no-insurance cases. General equilibrium effects will be studied separately.

In [ ]:
damage_map_compar_shareinc_insur.loc[
    damage_map_compar_shareinc_insur['max_val_' + 'formal'] == 0,
    'max_val_' + 'formal'] = np.nan
fig = px.choropleth_mapbox(
    damage_map_compar_shareinc_insur,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='max_val_' + 'formal',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'max_val_' + 'formal': 'Total change',
            'max_content_' + 'formal': 'o.w. content',
            'locations': 'Pixel ID',
            'flood_type_' + 'formal': 'Flood type',
            'nb_households_' + 'formal': 'Nb of households',
            'incgroup_' + 'formal': 'Dominant inc. group',
            'net_income_' + 'formal': 'Net income',
            'rent_' + 'formal' + '_pct': '% change in rent'},
    title='Flood damage surplus from no insurance in ' + 'formal'
    + ' housing (as share of net income)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'flood_type_' + 'formal': True,
                'nb_households_' + 'formal': ':,.0f',
                'incgroup_' + 'formal': ':,.0f',
                'rent_' + 'formal' + '_pct': ':+.2%',
                'net_income_' + 'formal': ':,.0f',
                'max_val_' + 'formal': ':+.2%',
                'max_content_' + 'formal': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

In [ ]:
damage_map_compar_shareinc_insur.loc[
    damage_map_compar_shareinc_insur['max_val_' + 'informal'] == 0,
    'max_val_' + 'informal'] = np.nan
fig = px.choropleth_mapbox(
    damage_map_compar_shareinc_insur,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='max_val_' + 'informal',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'max_val_' + 'informal': 'Total change',
            'max_content_' + 'informal': 'o.w. content',
            'locations': 'Pixel ID',
            'flood_type_' + 'informal': 'Flood type',
            'nb_households_' + 'informal': 'Nb of households',
            'incgroup_' + 'informal': 'Dominant inc. group',
            'net_income_' + 'informal': 'Net income',
            'rent_' + 'informal' + '_pct': '% change in rent'},
    title='Flood damage surplus from no insurance in ' + 'informal'
    + ' housing (as share of net income)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'flood_type_' + 'informal': True,
                'nb_households_' + 'informal': ':,.0f',
                'incgroup_' + 'informal': ':,.0f',
                'rent_' + 'informal' + '_pct': ':+.2%',
                'net_income_' + 'informal': ':,.0f',
                'max_val_' + 'informal': ':+.2%',
                'max_content_' + 'informal': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

In [ ]:
damage_map_compar_shareinc_insur.loc[
    damage_map_compar_shareinc_insur['max_val_' + 'backyard'] == 0,
    'max_val_' + 'backyard'] = np.nan
fig = px.choropleth_mapbox(
    damage_map_compar_shareinc_insur,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='max_val_' + 'backyard',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'max_val_' + 'backyard': 'Total change',
            'max_content_' + 'backyard': 'o.w. content',
            'locations': 'Pixel ID',
            'flood_type_' + 'backyard': 'Flood type',
            'nb_households_' + 'backyard': 'Nb of households',
            'incgroup_' + 'backyard': 'Dominant inc. group',
            'net_income_' + 'backyard': 'Net income',
            'rent_' + 'backyard' + '_pct': '% change in rent'},
    title='Flood damage surplus from no insurance in ' + 'backyard'
    + ' housing (as share of net income)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'flood_type_' + 'backyard': True,
                'nb_households_' + 'backyard': ':,.0f',
                'incgroup_' + 'backyard': ':,.0f',
                'rent_' + 'backyard' + '_pct': ':+.2%',
                'net_income_' + 'backyard': ':,.0f',
                'max_val_' + 'backyard': ':+.2%',
                'max_content_' + 'backyard': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

We do not show plots for formal subsidized housing as it is exogenous in our model, hence no changes occur across scenarios.

As expected, flood damage surplus is globally positive in the no-insurance case. The corresponding net income loss globally follows the spatial distribution of total absolute damages for formal private housing, with a relative peak East of the airport where income group 2 (the second poorest) lives, and a relative trough near Rosebank where income group 4 (the richest) lives.

Households living in formal private housing also seem to suffer the most (in relative terms) from flood damages. This should be nuanced however: although both structure and content damages have been included for the sake of comparability across maps, households living in formal private housing directly pay for content damages only in our model (as damages on structures are internalized by developers who own buildings). Taking only this margin into account, a quick look shows that damage exposure is in fact comparable with households living in informal settlements. Still, changes in rents would capture part of structural damages in the formal private sector, so that we can take the damage map as an upper bound in this case.

Damages concentrate in the area East of Driftsands Nature Reserve for informal settlements, where income group 1 (the poorest) lives and flood risk exposure is important. The same analysis applies for informal backyards located North and South of the reserve (with an order of magnitude below).

Fluvial flood risks remain the most important damage factor, even in relative terms.

#### Summay welfare statistics

To simplify welfare intepretation of the above, we plot the aggregate distribution of relative flood damages per income group. Since there is a high mass of households who do not live in flood-prone areas, the graphs only consider the subpopulation living in areas with a positive risk. In the interest of space, we only show results for fluvial flood risks (since they are the most impactful).

In [ ]:
dist_sum = px.histogram(
    damage_maps_shareinc_insur.loc[
        damage_maps_shareinc_insur[
            'sum_' + 'fluvialu' + '_' + '1'] > 0],
    x='sum_' + 'fluvialu' + '_' + '1',
    y='nb_households_' + '1',
    color='insur',
    labels={'sum_' + 'fluvialu' + '_' + '1': 'Share of net income',
            'nb_households_' + '1': 'nb of households',
            'insur': 'w/ insurance'},
    barmode='group',
    hover_data={'sum_' + 'fluvialu' + '_' + '1': False},
    title='Distribution of flood damages among income group '
    + '1' + ' (as share of net income)',
    template='plotly_white')
dist_sum.update_layout(
    yaxis_title='Total nb of households'
)

In [ ]:
dist_sum = px.histogram(
    damage_maps_shareinc_insur.loc[
        damage_maps_shareinc_insur[
            'sum_' + 'fluvialu' + '_' + '2'] > 0],
    x='sum_' + 'fluvialu' + '_' + '2',
    y='nb_households_' + '2',
    color='insur',
    labels={'sum_' + 'fluvialu' + '_' + '2': 'Share of net income',
            'nb_households_' + '2': 'nb of households',
            'insur': 'w/ insurance'},
    barmode='group',
    hover_data={'sum_' + 'fluvialu' + '_' + '2': False},
    title='Distribution of flood damages among income group '
    + '2' + ' (as share of net income)',
    template='plotly_white')
dist_sum.update_layout(
    yaxis_title='Total nb of households'
)

In [ ]:
dist_sum = px.histogram(
    damage_maps_shareinc_insur.loc[
        damage_maps_shareinc_insur[
            'sum_' + 'fluvialu' + '_' + '3'] > 0],
    x='sum_' + 'fluvialu' + '_' + '3',
    y='nb_households_' + '3',
    color='insur',
    labels={'sum_' + 'fluvialu' + '_' + '3': 'Share of net income',
            'nb_households_' + '3': 'nb of households',
            'insur': 'w/ insurance'},
    barmode='group',
    hover_data={'sum_' + 'fluvialu' + '_' + '3': False},
    title='Distribution of flood damages among income group '
    + '3' + ' (as share of net income)',
    template='plotly_white')
dist_sum.update_layout(
    yaxis_title='Total nb of households'
)

In [ ]:
dist_sum = px.histogram(
    damage_maps_shareinc_insur.loc[
        damage_maps_shareinc_insur[
            'sum_' + 'fluvialu' + '_' + '4'] > 0],
    x='sum_' + 'fluvialu' + '_' + '4',
    y='nb_households_' + '4',
    color='insur',
    labels={'sum_' + 'fluvialu' + '_' + '4': 'Share of net income',
            'nb_households_' + '4': 'nb of households',
            'insur': 'w/ insurance'},
    barmode='group',
    hover_data={'sum_' + 'fluvialu' + '_' + '4': False},
    title='Distribution of flood damages among income group '
    + '4' + ' (as share of net income)',
    template='plotly_white')
dist_sum.update_layout(
    yaxis_title='Total nb of households'
)

Here is our interpretation:
- Overall exposure to flood risks decreases with income (conditional on income group population), even when flood risks are not taken into account (no-insurance case). This corresponds to the intuition that flood-prone areas are typically low-amenity areas located far from (high) income centres, as public services and economic activity historically developed away from them.
- More counter-intuitively, the tail of flood vulnerability does not necessarily becomes thinner as income increases. We see that as the outcome of two counteracting forces: on the one hand, the denominator effect of higher income is expected to offset the numerator effect of higher values in exposed structures and contents ; on the other, as marginal utility is decreasing with income, richer households may prove less sensitive to a change in incomes.
- The very long (but thin) tail of flood damages for income group 1 (with some households losing more than 100% of their income) is due to a few households living in formal subsidized housing that are located in flood-prone areas, as the cost of such housing structures is relatively high (compared to their income or the capital value of an informal "shack"). This is sustained in our model since we assume that all poor households would sort into such housing (when given the choice) due to their poor outside options. In practice, this would require the public planner to bail out such households. In any case, this does not affect our comparative statics, which is our key focus here (we recall that formal subsidized housing is exogenous).
- When including insurance, households typically reduce offset their vulnerability in high-damage brackets by moving to low-damage brackets.

This last point corresponds to the intuition we first have about insurance policy being welfare-improving. Although marginal utility of income (and income levels) are not the same for all households within one group, we provide some aggregate statistics to approximate the direct welfare impact of such policy (for fluvial flood risks only):
- Average damages go from 9.83% of net income (for 104,723 households) to 3.37% (for 87,582 households) for income group 1 (397,395 households in total) between the no-insurance and full-insurance scenarios. Taking the exposed population in the no-insurance case as baseline (to include benefits to households who left flood zones entirely), **this translates into a 7.01% gain in net income for 26% of the group population (or a 1.85% gain overall)**.
- Average damages go from 2.57% (for 32,789 households) to 0.60% (for 30,363 households) for income group 2 (172,435 households). **This translates into a 2.01% gain in net income for 19% of the group population (or a 0.38% gain overall)**.
- Average damages go from 3.08% (for 38,620 households) to 1.79% (for 36,593 households) for income group 3 (296,993 households). **This translates into a 1.38% gain in net income for 13% of the group population (or a 0.18% gain overall)**.
- Average damages go from 1.59% (for 22,624 households) to 0.76% (for 21,192 households) for income group 4 (162,676 households). **This translates into a 0.88% gain in net income for 14% of the group population (or a 0.12% gain overall)**.

As expected, the welfare impact of the insurance policy roughly decreases with income. This would need to be compared to the cost of implementing (at least partially) such policy for a full-fledged cost-benefit analysis.

#### Population moves and composition effect

As mentioned before, the above says nothing of the mix between extensive and intensive margin responses used to obtain those results. We therefore turn to these, under the understanding that we will only be able to make qualitative statements. Indeed, disentangling between both margins quantitatively would require to follow individual households across scenarios: since we do not know which households would move from a given flood zone to another, we do not know which part of the associated damage reduction is due to migration vs. consumption adaptation. We recall that those responses also affect welfare indirectly through general equilibrium effects, although we are not able to make quantitative statements about the relative importance of direct and indirect effects either.

That being said, let us start with the extensive margin response.

First, in the full-insurance case:

In [ ]:
equil_map_insur['lon'] = geo_grid.lon
equil_map_insur['lat'] = geo_grid.lat
equil_map_insur.loc[
    equil_map_insur['hh_tot'] == 0, 'hh_tot'] = np.nan

fig = px.choropleth_mapbox(
    equil_map_insur,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='hh_tot',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'hh_formal': 'Formal private',
            'formal_incgroup': 'Inc. group (formal)',
            'hh_subsidized': 'Formal subsidized',
            'subsidized_incgroup': 'Inc. group (subsidized)',
            'hh_informal': 'Informal settlements',
            'informal_incgroup': 'Inc. group (informal)',
            'hh_backyard': 'Informal backyards',
            'backyard_incgroup': 'Inc. group (backyard)',
            'hh_tot': 'Total', 'locations': 'Pixel ID'},
    title='Estimated number of households',
    color_continuous_scale="Reds",
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'hh_formal': ':,.0f',
                'formal_incgroup': ':,.0f',
                'hh_subsidized': ':,.0f',
                'subsidized_incgroup': ':,.0f',
                'hh_informal': ':,.0f',
                'informal_incgroup': ':,.0f',
                'hh_backyard': ':,.0f',
                'backyard_incgroup': ':,.0f',
                'hh_tot': ':,.0f'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

Then, in the no-insurance case:

In [ ]:
equil_map_noinsur['lon'] = geo_grid.lon
equil_map_noinsur['lat'] = geo_grid.lat
equil_map_noinsur.loc[
    equil_map_noinsur['hh_tot'] == 0, 'hh_tot'] = np.nan

fig = px.choropleth_mapbox(
    equil_map_noinsur,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='hh_tot',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'hh_formal': 'Formal private',
            'formal_incgroup': 'Inc. group (formal)',
            'hh_subsidized': 'Formal subsidized',
            'subsidized_incgroup': 'Inc. group (subsidized)',
            'hh_informal': 'Informal settlements',
            'informal_incgroup': 'Inc. group (informal)',
            'hh_backyard': 'Informal backyards',
            'backyard_incgroup': 'Inc. group (backyard)',
            'hh_tot': 'Total', 'locations': 'Pixel ID'},
    title='Estimated number of households',
    color_continuous_scale="Reds",
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'hh_formal': ':,.0f',
                'formal_incgroup': ':,.0f',
                'hh_subsidized': ':,.0f',
                'subsidized_incgroup': ':,.0f',
                'hh_informal': ':,.0f',
                'informal_incgroup': ':,.0f',
                'hh_backyard': ':,.0f',
                'backyard_incgroup': ':,.0f',
                'hh_tot': ':,.0f'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

Finally, comparing the two:

In [ ]:
equil_map_compar_insur.loc[
    equil_map_compar_insur['hh_tot'] == 0, 'hh_tot'] = np.nan
fig = px.choropleth_mapbox(
    equil_map_compar_insur,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='hh_tot',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'hh_formal': 'Formal private',
            'formal_incgroup': 'Inc. group (formal)',
            'hh_subsidized': 'Formal subsidized',
            'subsidized_incgroup': 'Inc. group (subsidized)',
            'hh_informal': 'Informal settlements',
            'informal_incgroup': 'Inc. group (informal)',
            'hh_backyard': 'Informal backyards',
            'backyard_incgroup': 'Inc. group (backyard)',
            'hh_tot': 'Total change', 'hh_tot_pct': '% change',
            'locations': 'Pixel ID'},
    title='Evolution of number of households under no insurance',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'hh_formal': ':,.0f',
                'formal_incgroup': ':,.0f',
                'hh_subsidized': ':,.0f',
                'subsidized_incgroup': ':,.0f',
                'hh_informal': ':,.0f',
                'informal_incgroup': ':,.0f',
                'hh_backyard': ':,.0f',
                'backyard_incgroup': ':,.0f',
                'hh_tot': ':,.0f',
                'hh_tot_pct': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

Compared to previous versions of *NEDUM-2D*, we manage to predict well the high density in the Cape Flats (mostly through informal settlements in Khayelitsha and the Crossroads, but also formal subsidized housing in Delft and Nyanga), but slightly underestimate the density in the City Bowl. We do have a better fit for in-between neighborhoods however. The maps could be split across income groups or housing types, but we chose to display this information as hover data in the interest of space.

Regarding the extensive margin response, we see that, when introducing insurance:
- Most ouflows occur in highly vulnerable informal settlements East of Driftsands Nature Reserve (with some areas being completely abandoned), with an isolated move North of Paarlberg Nature Reserve. The effect is more dispersed, but nonetheless important, in formal private housing areas. Most people leaving the Cape Flats belong to income groups 1 and 2, while people from other suburbs tend to belong to income group 3.
- Inflows are more dispersed across Khayelitsha and in-between neighbourhoods (informal settlements and some formal private housing), plus isolated moves into Stellenbosch, Grabouw, and Atlantis (informal settlements).

Although we cannot follow individual households, here is a tentative interpretation. Poor households (most of them living in informal settlements) are very constrained. Therefore, they may need to move far away from their original locations to offset their losses, hence the high dispersion observed. Richer households, in contrast, may afford to move closer to their original locations.

Back to the welfare statements, this allows us to state that the extensive margin plays a significant role in observed damage reduction (when introducing insurance) for the two poorest income groups, but less so for the richest ones.

Also note that, due to the strength of financial constraints, flood risks are rarely big enough to change the income hierarchy of places. Still, it may happen that some income groups replace other ones across scenarios. Since it actually happens in very few flood zones, we do not show this composition effect in the comparison map above (for the sake of readability): we only show hover data for the dominant income group in the no-insurance scenario, and the dominant one in the full-insurance scenario only when nobody lives in the area under no insurance. Likewise, in the comparison map showing flood damages as a share of net income (where we just show hover data for the no-insurance case), the value of changes in such places would be biased by this effect: again, this is not key.

#### Spatial sorting determinants and consumption adaptation

Let us now turn to the determinants of observed spatial sorting, which will also explain the intensive margin response in terms of consumption adaptation. To do so, let us look at the evolution of rents across scenarios.

In [ ]:
equil_map_compar_insur.loc[
    equil_map_compar_insur['rent_' + 'formal'] == 0,
    'rent_' + 'formal'] = np.nan
fig = px.choropleth_mapbox(
    equil_map_compar_insur,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='rent_' + 'formal',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'locations': 'Pixel ID', 'lon': 'Lon.', 'lat': 'Lat.',
            'hh_' + 'formal' + '_pct': 'Nb of HHs (% change)',
            'formal' + '_incgroup': 'Inc. group',
            'dwelling_size_' + 'formal' + '_pct':
                'Dwelling size (% change)',
            'flood_deprec_struct_' + 'formal':
                'Struct. deprec. from floods',
            'flood_deprec_content_' + 'formal':
                'Content deprec. from floods',
            'rent_' + 'formal': 'Total change',
            'rent_' + 'formal' + '_pct': '% change'},
    title='Evolution of annual rent / m² under no insurance in '
    + 'formal' + ' housing (in rands, 2011)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'hh_' + 'formal' + '_pct': ':+.2%',
                'formal' + '_incgroup': ':,.0f',
                'dwelling_size_' + 'formal' + '_pct': ':+.2%',
                'flood_deprec_struct_' + 'formal': ':.2%',
                'flood_deprec_content_' + 'formal': ':.2%',
                'rent_' + 'formal': ':,.0f',
                'rent_' + 'formal' + '_pct': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

In [ ]:
equil_map_compar_insur.loc[
    equil_map_compar_insur['rent_' + 'informal'] == 0,
    'rent_' + 'informal'] = np.nan
fig = px.choropleth_mapbox(
    equil_map_compar_insur,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='rent_' + 'informal',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'locations': 'Pixel ID', 'lon': 'Lon.', 'lat': 'Lat.',
            'hh_' + 'informal' + '_pct': 'Nb of HHs (% change)',
            'informal' + '_incgroup': 'Inc. group',
            'dwelling_size_' + 'informal' + '_pct':
                'Dwelling size (% change)',
            'flood_deprec_struct_' + 'informal':
                'Struct. deprec. from floods',
            'flood_deprec_content_' + 'informal':
                'Content deprec. from floods',
            'rent_' + 'informal': 'Total change',
            'rent_' + 'informal' + '_pct': '% change'},
    title='Evolution of annual rent / m² under no insurance in '
    + 'informal' + ' housing (in rands, 2011)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'hh_' + 'informal' + '_pct': ':+.2%',
                'informal' + '_incgroup': ':,.0f',
                'dwelling_size_' + 'informal' + '_pct': ':+.2%',
                'flood_deprec_struct_' + 'informal': ':.2%',
                'flood_deprec_content_' + 'informal': ':.2%',
                'rent_' + 'informal': ':,.0f',
                'rent_' + 'informal' + '_pct': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

In [ ]:
equil_map_compar_insur.loc[
    equil_map_compar_insur['rent_' + 'backyard'] == 0,
    'rent_' + 'backyard'] = np.nan
fig = px.choropleth_mapbox(
    equil_map_compar_insur,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='rent_' + 'backyard',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'locations': 'Pixel ID', 'lon': 'Lon.', 'lat': 'Lat.',
            'hh_' + 'backyard' + '_pct': 'Nb of HHs (% change)',
            'backyard' + '_incgroup': 'Inc. group',
            'dwelling_size_' + 'backyard' + '_pct':
                'Dwelling size (% change)',
            'flood_deprec_struct_' + 'backyard':
                'Struct. deprec. from floods',
            'flood_deprec_content_' + 'backyard':
                'Content deprec. from floods',
            'rent_' + 'backyard': 'Total change',
            'rent_' + 'backyard' + '_pct': '% change'},
    title='Evolution of annual rent / m² under no insurance in '
    + 'backyard' + ' housing (in rands, 2011)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'hh_' + 'backyard' + '_pct': ':+.2%',
                'backyard' + '_incgroup': ':,.0f',
                'dwelling_size_' + 'backyard' + '_pct': ':+.2%',
                'flood_deprec_struct_' + 'backyard': ':.2%',
                'flood_deprec_content_' + 'backyard': ':.2%',
                'rent_' + 'backyard': ':,.0f',
                'rent_' + 'backyard' + '_pct': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

Again, we do not plot formal subsidized housing as it rented out / sold for free in the model.

We see that the areas with the biggest changes in rents roughly correspond to inflow/outflow areas in the Cape Flats and distant suburbs. This makes sense as market rents reflect the willingness to pay of the highest-bidding income groups in the model. As outflow areas typically are high-risk flood zones (compared to inflow areas which typically are not flood zones), this reflects the updated valuations after introducing insurance (note that % change is set equal to zero in hover data when there are no households to compare to in one of the scenarios, as in previous maps).

Flood zones closer to the city center (and in some coastal locations) typically feature smaller (yet significant) changes in rents, even when they are as exposed as zones located farther away. This corresponds to the intuition already mentioned that, since those typically are high-value areas, demand would not fall below a certain level, all the more so as households living there can afford to pay for flood damages. Consequently, they do not showcase substantial population flows. Population flows and rent changes sometimes even go in the opposite direction, as some households are willing to take advantage of lower rents. Likewise, if a fall in rent typically translates into a rise in housing consumption, it is not always the case (if households want to save on flood damages, for instance).

In fact, market rents summarize both extensive and intensive margin responses that interact with each other, hence their tricky interpretation. Still, we can conclude that the intensive margin plays a more important (if ambiguous) role in observed damage reduction (when introducing insurance) for the two richest income groups, compared to the two poorest ones.

### Climate change

#### Introduction

We now take the full-insurance scenario as our baseline and simulate the effects of climate change. For coastal flood risks, we rely on specific flood maps provided by DELTARES, based upon IPCC's RCP (Representative Concentration Pathway) 8.5 scenario projected onto 2050. Since FATHOM does not provide similar flood maps for rainwater flood risks, we consider dummy scenarios where we multiply all yearly probabilities by 2 (that is, we divide return periods by 2) for such flood types. Although we do not simulate the changes in affected areas in that case, we believe it is a good enough approximation since they are not likely to change much (as is the case when incorporating sea-level rise in coastal flood maps). Then, we picked the likelihood ratio to simulate changes in rainwater flood risks aligned with changes in coastal flood risks. This also appears when plotting the changes in flood capital depreciation, as we show below (for content damages).

In [ ]:
equil_map_compar_cchange.loc[
    equil_map_compar_cchange['flood_deprec_content_formal'] == 0,
    'flood_deprec_content_formal'] = np.nan
fig = px.choropleth_mapbox(
    equil_map_compar_cchange,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='flood_deprec_content_formal',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-terrain',
    opacity=0.5,
    labels={'locations': 'Pixel ID', 'lon': 'Lon.', 'lat': 'Lat.',
            'flood_deprec_content_formal': 'Deprec. (% change)'},
    title='Estimated change in fraction of capital destroyed due to floods'
    + ' with climate change (content damages)',
    color_continuous_scale="Reds",
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'flood_deprec_content_formal': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

It should be noted that those scenarios are generally thought of as being relatively pessimistic. Also, we are not focusing on the accuracy of climate change predictions per se, but rather on the general trends climate change may generate in the city. As such, we are not too concerned about the choice of our scenarios, as long as they are coherent.

#### Aggregate damages

As before, we start by plotting aggregate damages per flood type across the two scenarios. Note that we do not consider a climate change scenario without insurance, since we expect it to yield similar patterns as the standard no-insurance case, but with bigger damage values. Besides, we are more specifically interested in how climate change may affect spatial sorting in the long run, which requires agents to take flood risks into account when making their decisions.

In [ ]:
# We store the dimension names to be used when superimposing the plots
newnames_nocchange = {'Structures': 'Structures (w/o/ c. change)',
                      'Contents': 'Contents (w/o/ c. change)'}
newnames_cchange = {'Structures': 'Structures (w/ c. change)',
                    'Contents': 'Contents (w/ c. change)'}

In [ ]:
# We start with fluvial floods

fig_fluvialu_nocchange = px.bar(
    dict_df_agg_damages_cchange['fluvialu_nocchange'],
    x=dict_df_agg_damages_cchange['fluvialu_nocchange'].index,
    y=dict_df_agg_damages_cchange['fluvialu_nocchange'].columns,
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Set1,
    title='Estimated annual damages from fluvial floods (in M rands, 2011)',
    labels={'index': 'Housing type', 'value': 'Annual damages',
            'variable': 'Damage type'},
    opacity=0.8,
    template='plotly_white')

fig_fluvialu_nocchange.for_each_trace(
    lambda t: t.update(
        name=newnames_nocchange[t.name],
        legendgroup=newnames_nocchange[t.name],
        hovertemplate=t.hovertemplate.replace(
         t.name, newnames_nocchange[t.name])
    ))

fig_fluvialu_cchange = px.bar(
    dict_df_agg_damages_cchange['fluvialu_cchange'],
    x=dict_df_agg_damages_cchange['fluvialu_cchange'].index,
    y=dict_df_agg_damages_cchange['fluvialu_cchange'].columns,
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Pastel1,
    title='Estimated annual damages from fluvial floods (in M rands, 2011)',
    labels={'index': 'Housing type', 'value': 'Annual damages',
            'variable': 'Damage type'},
    opacity=0.8,
    template='plotly_white')

fig_fluvialu_cchange.for_each_trace(
    lambda t: t.update(
        name=newnames_cchange[t.name],
        legendgroup=newnames_cchange[t.name],
        hovertemplate=t.hovertemplate.replace(
         t.name, newnames_cchange[t.name])
    ))

fig_fluvialu = go.Figure(fig_fluvialu_cchange)
fig_fluvialu = fig_fluvialu.add_traces(fig_fluvialu_nocchange.data)
fig_fluvialu.show()

In [ ]:
# Then pluvial floods

fig_pluvial_nocchange = px.bar(
    dict_df_agg_damages_cchange['pluvial_nocchange'],
    x=dict_df_agg_damages_cchange['pluvial_nocchange'].index,
    y=dict_df_agg_damages_cchange['pluvial_nocchange'].columns,
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Set1,
    title='Estimated annual damages from pluvial floods (in M rands, 2011)',
    labels={'index': 'Housing type', 'value': 'Annual damages',
            'variable': 'Damage type'},
    opacity=0.8,
    template='plotly_white')

fig_pluvial_nocchange.for_each_trace(
    lambda t: t.update(
        name=newnames_nocchange[t.name],
        legendgroup=newnames_nocchange[t.name],
        hovertemplate=t.hovertemplate.replace(
         t.name, newnames_nocchange[t.name])
    ))

fig_pluvial_cchange = px.bar(
    dict_df_agg_damages_cchange['pluvial_cchange'],
    x=dict_df_agg_damages_cchange['pluvial_cchange'].index,
    y=dict_df_agg_damages_cchange['pluvial_cchange'].columns,
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Pastel1,
    title='Estimated annual damages from pluvial floods (in M rands, 2011)',
    labels={'index': 'Housing type', 'value': 'Annual damages',
            'variable': 'Damage type'},
    opacity=0.8,
    template='plotly_white')

fig_pluvial_cchange.for_each_trace(
    lambda t: t.update(
        name=newnames_cchange[t.name],
        legendgroup=newnames_cchange[t.name],
        hovertemplate=t.hovertemplate.replace(
         t.name, newnames_cchange[t.name])
    ))

fig_pluvial = go.Figure(fig_pluvial_cchange)
fig_pluvial = fig_pluvial.add_traces(fig_pluvial_nocchange.data)
fig_pluvial.show()

In [ ]:
# Finally coastal floods

fig_coastal_nocchange = px.bar(
    dict_df_agg_damages_cchange['coastal_nocchange'],
    x=dict_df_agg_damages_cchange['coastal_nocchange'].index,
    y=dict_df_agg_damages_cchange['coastal_nocchange'].columns,
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Set1,
    title='Estimated annual damages from coastal floods (in M rands, 2011)',
    labels={'index': 'Housing type', 'value': 'Annual damages',
            'variable': 'Damage type'},
    opacity=0.8,
    template='plotly_white')

fig_coastal_nocchange.for_each_trace(
    lambda t: t.update(
        name=newnames_nocchange[t.name],
        legendgroup=newnames_nocchange[t.name],
        hovertemplate=t.hovertemplate.replace(
         t.name, newnames_nocchange[t.name])
    ))

fig_coastal_cchange = px.bar(
    dict_df_agg_damages_cchange['coastal_cchange'],
    x=dict_df_agg_damages_cchange['coastal_cchange'].index,
    y=dict_df_agg_damages_cchange['coastal_cchange'].columns,
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Pastel1,
    title='Estimated annual damages from coastal floods (in M rands, 2011)',
    labels={'index': 'Housing type', 'value': 'Annual damages',
            'variable': 'Damage type'},
    opacity=0.8,
    template='plotly_white')

fig_coastal_cchange.for_each_trace(
    lambda t: t.update(
        name=newnames_cchange[t.name],
        legendgroup=newnames_cchange[t.name],
        hovertemplate=t.hovertemplate.replace(
         t.name, newnames_cchange[t.name])
    ))

fig_coastal = go.Figure(fig_coastal_cchange)
fig_coastal = fig_coastal.add_traces(fig_coastal_nocchange.data)
fig_coastal.show()

First, we observe that the biggest change now occurs for pluvial flood risks, and not fluvial ones. The explanation is the same as before, but reversed: since pluvial flood zones are more spread out, climate change there is harder to avoid. Again, coastal flood zones fall in between as they typically are high-value areas where households prefer to adapt on the intensive rather than the extensive margin.

#### Spatial distribution of absolute damages

We go on with the spatial distribution of absolute damages.

In [ ]:
damage_map_compar_cchange.loc[
    damage_map_compar_cchange['max_val'] == 0, 'max_val'] = np.nan
fig = px.choropleth_mapbox(
    damage_map_compar_cchange,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='max_val',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'max_val': 'Total change', 'max_val_pct': '% change',
            'locations': 'Pixel ID',
            'flood_type': 'Flood type',
            'damage_formal': 'Formal private',
            'damage_formal_pct': '% change (formal)',
            'damage_subsidized': 'Formal subsidized',
            'damage_subsidized_pct': '% change (subsidized)',
            'damage_informal': 'Informal settlements',
            'damage_informal_pct': '% change (informal)',
            'damage_backyard': 'Informal backyards',
            'damage_backyard_pct': '% change (backyards)'},
    title='Flood damage surplus from climate change (in rands, 2011)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f', 'flood_type': True,
                'damage_formal': ':,.0f', 'damage_formal_pct': ':+.0%',
                'damage_subsidized': ':,.0f', 'damage_subsidized_pct': ':+.0%',
                'damage_informal': ':,.0f', 'damage_informal_pct': ':+.0%',
                'damage_backyard': ':,.0f', 'damage_backyard_pct': ':+.0%',
                'max_val': True, 'max_val_pct': ':+.0%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

We observe that the change across scenarios is more widely spread (but also smaller per unit of land) than in the previous use case (which relates to the above interpretation, as pluvial flood risks are also less severe than fluvial ones).

#### Spatial distribution of relative damages

As before, we also plot the spatial distribution of flood damages as a share of net income, to get an idea of direct welfare effects.

However, contrary to the previous use case, the spatial indifference condition holds in the two scenarios, hence there is no need to re-compute utilities. As we have seen, comparing the utility levels across scenarios provide a first-order approximation of general equilibrium welfare effects (understood as equivalent variation here). Under climate change, we obtain that:
- Households from income group 1 only see their welfare marginally impacted (in a non-statistically significant way)
- Households from income group 2 are 0.07% worse off
- Households from income group 3 are 0.05% worse off
- Households from income group 4 are 0.05% worse off

Several comments are in order:
- The effects are negative, yet very small. This is not to downgrade the overall importance of climate change. This merely points to the fact that, when only considering flood monetary impacts on the housing market, climate change may be a secondary issue **once we allow agents to arbitrage in space**. This means that losses can be minimized once agents adapt to climate change and behave in an optimal way. **However, this says nothing of the time span required to reach such equilibrium**.
- We also caution the reader against taking those figures at face value. Indeed, we do not know the sign or the magnitude of second-order effects induced by non-homotheticity (households spend a smaller share of their budget on housing when their income increases) and taste shocks (households have idiosyncratic preferences for locations that capture the residual factors needed to obtain a unique equilibrium).
- Heterogeneous effects across income groups depend on their relative spatial distribution that we are going to study now

In [ ]:
damage_map_compar_shareinc_cchange.loc[
    damage_map_compar_shareinc_cchange['max_val_' + 'formal'] == 0,
    'max_val_' + 'formal'] = np.nan
fig = px.choropleth_mapbox(
    damage_map_compar_shareinc_cchange,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='max_val_' + 'formal',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'max_val_' + 'formal': 'Total change',
            'max_content_' + 'formal': 'o.w. content',
            'locations': 'Pixel ID',
            'flood_type_' + 'formal': 'Flood type',
            'nb_households_cc_' + 'formal':
                'Nb of households (w/ c.c.)',
            'incgroup_cc_' + 'formal': 'Inc. group (w/ c.c.)',
            'net_income_cc_' + 'formal': 'Net income (w/ c.c.)',
            'rent_' + 'formal' + '_pct': 'Rent (% change)'},
    title='Flood damage surplus from climate change in ' + 'formal'
    + ' housing (as share of net income)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'flood_type_' + 'formal': True,
                'nb_households_cc_' + 'formal': ':,.0f',
                'incgroup_cc_' + 'formal': ':,.0f',
                'net_income_cc_' + 'formal': ':,.0f',
                'rent_' + 'formal' + '_pct': ':+.2%',
                'max_val_' + 'formal': ':+.2%',
                'max_content_' + 'formal': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

In [ ]:
damage_map_compar_shareinc_cchange.loc[
    damage_map_compar_shareinc_cchange['max_val_' + 'informal'] == 0,
    'max_val_' + 'informal'] = np.nan
fig = px.choropleth_mapbox(
    damage_map_compar_shareinc_cchange,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='max_val_' + 'informal',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'max_val_' + 'informal': 'Total change',
            'max_content_' + 'informal': 'o.w. content',
            'locations': 'Pixel ID',
            'flood_type_' + 'informal': 'Flood type',
            'nb_households_cc_' + 'informal':
                'Nb of households (w/ c.c.)',
            'incgroup_cc_' + 'informal': 'Inc. group (w/ c.c.)',
            'net_income_cc_' + 'informal': 'Net income (w/ c.c.)',
            'rent_' + 'informal' + '_pct': 'Rent (% change)'},
    title='Flood damage surplus from climate change in ' + 'informal'
    + ' housing (as share of net income)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'flood_type_' + 'informal': True,
                'nb_households_cc_' + 'informal': ':,.0f',
                'incgroup_cc_' + 'informal': ':,.0f',
                'net_income_cc_' + 'informal': ':,.0f',
                'rent_' + 'informal' + '_pct': ':+.2%',
                'max_val_' + 'informal': ':+.2%',
                'max_content_' + 'informal': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

In [ ]:
damage_map_compar_shareinc_cchange.loc[
    damage_map_compar_shareinc_cchange['max_val_' + 'backyard'] == 0,
    'max_val_' + 'backyard'] = np.nan
fig = px.choropleth_mapbox(
    damage_map_compar_shareinc_cchange,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='max_val_' + 'backyard',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'max_val_' + 'backyard': 'Total change',
            'max_content_' + 'backyard': 'o.w. content',
            'locations': 'Pixel ID',
            'flood_type_' + 'backyard': 'Flood type',
            'nb_households_cc_' + 'backyard':
                'Nb of households (w/ c.c.)',
            'incgroup_cc_' + 'backyard': 'Inc. group (w/ c.c.)',
            'net_income_cc_' + 'backyard': 'Net income (w/ c.c.)',
            'rent_' + 'backyard' + '_pct': 'Rent (% change)'},
    title='Flood damage surplus from climate change in ' + 'backyard'
    + ' housing (as share of net income)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'flood_type_' + 'backyard': True,
                'nb_households_cc_' + 'backyard': ':,.0f',
                'incgroup_cc_' + 'backyard': ':,.0f',
                'net_income_cc_' + 'backyard': ':,.0f',
                'rent_' + 'backyard' + '_pct': ':+.2%',
                'max_val_' + 'backyard': ':+.2%',
                'max_content_' + 'backyard': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

In [ ]:
damage_map_compar_shareinc_cchange.loc[
    damage_map_compar_shareinc_cchange['max_val_' + 'subsidized'] == 0,
    'max_val_' + 'subsidized'] = np.nan
fig = px.choropleth_mapbox(
    damage_map_compar_shareinc_cchange,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='max_val_' + 'subsidized',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'max_val_' + 'subsidized': 'Total change',
            'max_content_' + 'subsidized': 'o.w. content',
            'locations': 'Pixel ID',
            'flood_type_' + 'subsidized': 'Flood type',
            'nb_households_cc_' + 'subsidized':
                'Nb of households (w/ c.c.)',
            'incgroup_cc_' + 'subsidized': 'Inc. group (w/ c.c.)',
            'net_income_cc_' + 'subsidized': 'Net income (w/ c.c.)',
            'rent_' + 'subsidized' + '_pct': 'Rent (% change)'},
    title='Flood damage surplus from climate change in ' + 'subsidized'
    + ' housing (as share of net income)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'flood_type_' + 'subsidized': True,
                'nb_households_cc_' + 'subsidized': ':,.0f',
                'incgroup_cc_' + 'subsidized': ':,.0f',
                'net_income_cc_' + 'subsidized': ':,.0f',
                'rent_' + 'subsidized' + '_pct': ':+.2%',
                'max_val_' + 'subsidized': ':+.2%',
                'max_content_' + 'subsidized': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

Compared to the absolute damage map, we observe a relative peak in formal private relative damages South of Sunset Beach, and a relative trough near Rosebank and Claremont. It is simply because the first zone is populated by households from income group 3 and the second zone by households from income group 4.

In informal settlements, some damage values increase in Khayelitsha and North of Table Mountain National Park, but the biggest changes occur North of Table Bay Nature Reserve: it is likely that households move from one point to another with close characteristics but less exposed to floods.

In informal backyards, damage values are globally increasing: they concentrate in Khayelitsha, Delft, and Strand, where households are likely to be restricted in their movements. Another interpretation is that the increase in flood risks is not big enough to reach a tipping point that makes households migrate.

In formal subsidized housing, households are immobile for sure (and are not taken into account when computing utility levels), and bear a more significant share of flood damages because of the relative importance of structure costs there. Damages are distributed in a similar way as for informal backyards, which suggests that backyarding might be a way for poor households to cope with such increasing costs.

#### Summay welfare statistics

As before, we plot the aggregate distribution of relative damages, this time for pluvial flood risks, which play the dominant role.

In [ ]:
dist_sum = px.histogram(
    damage_maps_shareinc_cchange.loc[
        damage_maps_shareinc_cchange[
            'sum_' + 'pluvial' + '_' + '1'] > 0],
    x='sum_' + 'pluvial' + '_' + '1',
    y='nb_households_' + '1',
    color='cchange',
    labels={'sum_' + 'pluvial' + '_' + '1': 'Share of net income',
            'nb_households_' + '1': 'nb of households',
            'cchange': 'w/ c.change'},
    barmode='group',
    hover_data={'sum_' + 'pluvial' + '_' + '1': False},
    title='Distribution of flood damages among income group '
    + '1' + ' (as share of net income)',
    template='plotly_white')

dist_sum.update_layout(
    yaxis_title='Total nb of households'
)

In [ ]:
dist_sum = px.histogram(
    damage_maps_shareinc_cchange.loc[
        damage_maps_shareinc_cchange[
            'sum_' + 'pluvial' + '_' + '2'] > 0],
    x='sum_' + 'pluvial' + '_' + '2',
    y='nb_households_' + '2',
    color='cchange',
    labels={'sum_' + 'pluvial' + '_' + '2': 'Share of net income',
            'nb_households_' + '2': 'nb of households',
            'cchange': 'w/ c.change'},
    barmode='group',
    hover_data={'sum_' + 'pluvial' + '_' + '2': False},
    title='Distribution of flood damages among income group '
    + '2' + ' (as share of net income)',
    template='plotly_white')

dist_sum.update_layout(
    yaxis_title='Total nb of households'
)

In [ ]:
dist_sum = px.histogram(
    damage_maps_shareinc_cchange.loc[
        damage_maps_shareinc_cchange[
            'sum_' + 'pluvial' + '_' + '3'] > 0],
    x='sum_' + 'pluvial' + '_' + '3',
    y='nb_households_' + '3',
    color='cchange',
    labels={'sum_' + 'pluvial' + '_' + '3': 'Share of net income',
            'nb_households_' + '3': 'nb of households',
            'cchange': 'w/ c.change'},
    barmode='group',
    hover_data={'sum_' + 'pluvial' + '_' + '3': False},
    title='Distribution of flood damages among income group '
    + '3' + ' (as share of net income)',
    template='plotly_white')

dist_sum.update_layout(
    yaxis_title='Total nb of households'
)

In [ ]:
dist_sum = px.histogram(
    damage_maps_shareinc_cchange.loc[
        damage_maps_shareinc_cchange[
            'sum_' + 'pluvial' + '_' + '4'] > 0],
    x='sum_' + 'pluvial' + '_' + '4',
    y='nb_households_' + '4',
    color='cchange',
    labels={'sum_' + 'pluvial' + '_' + '4': 'Share of net income',
            'nb_households_' + '4': 'nb of households',
            'cchange': 'w/ c.change'},
    barmode='group',
    hover_data={'sum_' + 'pluvial' + '_' + '4': False},
    title='Distribution of flood damages among income group '
    + '4' + ' (as share of net income)',
    template='plotly_white')

dist_sum.update_layout(
    yaxis_title='Total nb of households'
)

Compared to fluvial flood risks, there are more households exposed and the thail is both thinner and shorter (if not for formal subsidized housing damages). As expected, we see exposure shifting from low-damage brackets to high-damage brackets when introducing climate change.

We also provide some back-of-the-envelope calculation to approximate the direct welfare impact of climate change (for pluvial flood risks only):
- Average damages go from 0.67% of net income (for 366,265 households) to 1.34% (for 364,492 households) for income group 1 (397,395 households in total) between the no-climate change and the climate change scenarios. Taking the exposed population in the no-climate change case as baseline (to include benefits to households who left flood zones entirely), **this translates into a 0.67% loss in net income for 92% of the group population (or a 0.62% loss overall)**.
- Average damages go from 0.12% (for 166,655 households) to 0.23% (for 166,511 households) for income group 2 (172,435 households). **This translates into a 0.12% loss in net income for 97% of the group population (or a 0.11% loss overall)**.
- Average damages go from 0.10% (for 270,802 households) to 0.20% (for 270,786 households) for income group 3 (296,993 households). **This translates into a 0.10% loss in net income for 91% of the group population (or a 0.09% loss overall)**.
- Average damages go from 0.07% (for 128,747 households) to 0.13% (for 128,642 households) for income group 4 (162,676 households). **This translates into a 0.07% loss in net income for 79% of the group population (or a 0.05% loss overall)**.

Note how these losses are partly offset through general equilibrium effects if we follow the first-order approximation from last section. Also note that the order of magnitude of both effects is in line (the discrepancy for the poorest income group being due to exogenous formal subsidized dwellers being excluded from endogenous utility level computation).

#### Population moves and composition effect

Let us now dig into the extensive margin response.

In [ ]:
equil_map_compar_cchange.loc[
    equil_map_compar_cchange['hh_tot'] == 0, 'hh_tot'] = np.nan
fig = px.choropleth_mapbox(
    equil_map_compar_cchange,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='hh_tot',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'lon': 'Lon.', 'lat': 'Lat.',
            'hh_formal': 'Formal private',
            'formal_incgroup': 'Inc. group (formal)',
            'hh_subsidized': 'Formal subsidized',
            'subsidized_incgroup': 'Inc. group (subsidized)',
            'hh_informal': 'Informal settlements',
            'informal_incgroup': 'Inc. group (informal)',
            'hh_backyard': 'Informal backyards',
            'backyard_incgroup': 'Inc. group (backyard)',
            'hh_tot': 'Total change', 'hh_tot_pct': '% change',
            'locations': 'Pixel ID'},
    title='Evolution of number of households under climate change',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'hh_formal': ':,.0f',
                'formal_incgroup': ':,.0f',
                'hh_subsidized': ':,.0f',
                'subsidized_incgroup': ':,.0f',
                'hh_informal': ':,.0f',
                'informal_incgroup': ':,.0f',
                'hh_backyard': ':,.0f',
                'backyard_incgroup': ':,.0f',
                'hh_tot': ':,.0f',
                'hh_tot_pct': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

Main population outflows are spread out in specific locations in Stellenbosch, North of Table Mountain National Park, Athlone, Durbanville, and Bloubergstrand. Inflows concentrate in other areas of Athlone and Bloubergstrand. Overall, migrations remain limited in scope and do not seem to explain a significant share of damage changes under climate change.

#### Spatial sorting determinants and consumption adaptation

Let us now turn to the intensive margin response (again, we do not plot formal subsidized housing). 

In [ ]:
equil_map_compar_cchange.loc[
    equil_map_compar_cchange['rent_' + 'formal'] == 0,
    'rent_' + 'formal'] = np.nan
fig = px.choropleth_mapbox(
    equil_map_compar_cchange,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='rent_' + 'formal',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'locations': 'Pixel ID', 'lon': 'Lon.', 'lat': 'Lat.',
            'hh_' + 'formal' + '_pct': 'Nb of HHs (% change)',
            'formal' + '_incgroup': 'Inc. group',
            'dwelling_size_' + 'formal' + '_pct':
                'Dwelling size (% change)',
            'flood_deprec_struct_' + 'formal':
                'Struct. deprec. (% change)',
            'flood_deprec_content_' + 'formal':
                'Content deprec. (% change)',
            'rent_' + 'formal': 'Rent (total change)',
            'rent_' + 'formal' + '_pct': '% change'},
    title='Evolution of annual rent / m² under climate change in '
    + 'formal' + ' housing (in rands, 2011)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'hh_' + 'formal' + '_pct': ':+.2%',
                'formal' + '_incgroup': ':,.0f',
                'dwelling_size_' + 'formal' + '_pct': ':+.2%',
                'flood_deprec_struct_' + 'formal': ':+.2%',
                'flood_deprec_content_' + 'formal': ':+.2%',
                'rent_' + 'formal': ':,.0f',
                'rent_' + 'formal' + '_pct': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

In [ ]:
equil_map_compar_cchange.loc[
    equil_map_compar_cchange['rent_' + 'informal'] == 0,
    'rent_' + 'informal'] = np.nan
fig = px.choropleth_mapbox(
    equil_map_compar_cchange,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='rent_' + 'informal',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'locations': 'Pixel ID', 'lon': 'Lon.', 'lat': 'Lat.',
            'hh_' + 'informal' + '_pct': 'Nb of HHs (% change)',
            'informal' + '_incgroup': 'Inc. group',
            'dwelling_size_' + 'informal' + '_pct':
                'Dwelling size (% change)',
            'flood_deprec_struct_' + 'informal':
                'Struct. deprec. (% change)',
            'flood_deprec_content_' + 'informal':
                'Content deprec. (% change)',
            'rent_' + 'informal': 'Rent (total change)',
            'rent_' + 'informal' + '_pct': '% change'},
    title='Evolution of annual rent / m² under climate change in '
    + 'informal' + ' housing (in rands, 2011)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'hh_' + 'informal' + '_pct': ':+.2%',
                'informal' + '_incgroup': ':,.0f',
                'dwelling_size_' + 'informal' + '_pct': ':+.2%',
                'flood_deprec_struct_' + 'informal': ':+.2%',
                'flood_deprec_content_' + 'informal': ':+.2%',
                'rent_' + 'informal': ':,.0f',
                'rent_' + 'informal' + '_pct': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

In [ ]:
equil_map_compar_cchange.loc[
    equil_map_compar_cchange['rent_' + 'backyard'] == 0,
    'rent_' + 'backyard'] = np.nan
fig = px.choropleth_mapbox(
    equil_map_compar_cchange,
    geojson=geo_grid.geometry,
    locations=geo_grid.index,
    color='rent_' + 'backyard',
    center={"lat": -33.92345542582841, "lon": 18.434424141913478},
    zoom=9.25,
    mapbox_style='stamen-toner',
    opacity=0.75,
    labels={'locations': 'Pixel ID', 'lon': 'Lon.', 'lat': 'Lat.',
            'hh_' + 'backyard' + '_pct': 'Nb of HHs (% change)',
            'backyard' + '_incgroup': 'Inc. group',
            'dwelling_size_' + 'backyard' + '_pct':
                'Dwelling size (% change)',
            'flood_deprec_struct_' + 'backyard':
                'Struct. deprec. (% change)',
            'flood_deprec_content_' + 'backyard':
                'Content deprec. (% change)',
            'rent_' + 'backyard': 'Rent (total change)',
            'rent_' + 'backyard' + '_pct': '% change'},
    title='Evolution of annual rent / m² under climate change in '
    + 'backyard' + ' housing (in rands, 2011)',
    color_continuous_scale="Picnic",
    color_continuous_midpoint=0,
    template='plotly_white',
    hover_data={'lon': ':.2f', 'lat': ':.2f',
                'hh_' + 'backyard' + '_pct': ':+.2%',
                'backyard' + '_incgroup': ':,.0f',
                'dwelling_size_' + 'backyard' + '_pct': ':+.2%',
                'flood_deprec_struct_' + 'backyard': ':+.2%',
                'flood_deprec_content_' + 'backyard': ':+.2%',
                'rent_' + 'backyard': ':,.0f',
                'rent_' + 'backyard' + '_pct': ':+.2%'})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0})
fig.update_traces(marker_line_width=0)

In formal private housing, we observe big changes in rents in Athlone migration areas: households leave flood zones where rents decrease, and go to nearby unexposed places where rents increase. This masks less important (but still relevant) rent decreases East of the City Bowl and around Sunset Beach, following increases in flood risks. Those are not massive emigration areas, and households do seem to increase their housing consumption there: this explains the high damage values in absolute terms, at the same time as the low values in relative terms (rich households living there can afford to pay for flood damages). In other areas where rent does not move much, there is actually little response: households merely bear the changes in costs without adapting much.

In informal settlements, rent globally decreases, with troughs in Bloubergstrand and Khayelitsha. In the first zone, households move from one place to another so as to mitigate the rise in flood damages (as shown in the relative flood damage map). In the second zone, households do not seem to move much, and the rise in damages is mitigated by the fall in capital value (that is not compensated by a higher housing consumption). This is in line with the intuition that poor households are more sensitive to flood damages due to financial constraints.

Finally, in informal backyards, the fall in rents is more dispersed. Again, there does not seem to be a massive extensive margin response, and households essentially bear the increase in costs.

All in all, we observe the same types of adaptative behaviour as in the previous use case, with poor households who are more prone to mitigate the increase in flood damages (at the price of relatively bigger welfare loss), and a smaller role for the extensive margin response (compared to the intensive margin). 